# Actividad 4.3 Diagramas de voronoy
**Victor Misael Escalante Alvarado**

In [4]:

import numpy as np
import matplotlib.pyplot as plt
import math


## Funciones y codigo Previo

### Clases Usaddas para la triangulacion

In [3]:
# Clase para los puntos
class Punto:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def show(self):
        return [self.x, self.y]

# Clase para los segmentos
class Segmento:
    def __init__(self, A, B):
        # Asegura que el punto A sea siempre menor que B por consistencia
        if A.x < B.x or (A.x == B.x and A.y < B.y):
            self.A = A
            self.B = B
        else:
            self.A = B
            self.B = A

    def show(self):
        return [self.A.x, self.A.y, self.B.x, self.B.y]

    # Comprueba la igualdad entre dos segmentos en función de sus extremos
    def __eq__(self, other):
        return self.A == other.A and self.B == other.B

    # Genera un hash en función de los extremos
    def __hash__(self):
        return hash((self.A.x, self.A.y, self.B.x, self.B.y))

# Clase para los triángulos
class Triangulo:
    def __init__(self, puntos):
        self.puntos = puntos  # Lista de tres instancias de Punto
        self.lados = [Segmento(puntos[0], puntos[1]),
                      Segmento(puntos[1], puntos[2]),
                      Segmento(puntos[2], puntos[0])]


### Funciones de la tarea anterior

In [2]:


# Calcula circuncentro y circunradio
def calcular_circuncentro_y_radio(p1, p2, p3):
    D = 2 * (p1.x * (p2.y - p3.y) + p2.x * (p3.y - p1.y) + p3.x * (p1.y - p2.y))

    if D == 0: # Un determinante de 0 indica que no hay circumferencia
        return None, None

    # Circuncentro
    Ux = ((p1.x**2 + p1.y**2) * (p2.y - p3.y) +
          (p2.x**2 + p2.y**2) * (p3.y - p1.y) +
          (p3.x**2 + p3.y**2) * (p1.y - p2.y)) / D
    Uy = ((p1.x**2 + p1.y**2) * (p3.x - p2.x) +
          (p2.x**2 + p2.y**2) * (p1.x - p3.x) +
          (p3.x**2 + p3.y**2) * (p2.x - p1.x)) / D

    # Circunradio
    radio = math.sqrt((Ux - p1.x)**2 + (Uy - p1.y)**2)

    return Punto(Ux, Uy), radio

# Verifica si un punto está dentro o sobre una circunferencia
def punto_dentro_circunferencia(punto, centro, radio):
    distancia = math.sqrt((centro.x - punto.x)**2 + (centro.y - punto.y)**2) # Distancia del punto al circuncentro
    return distancia <= radio  # Retorna True si está dentro del radio

# Función para graficar
def graficar_triangulos_y_puntos(puntos, triangulos):
    fig, ax = plt.subplots()
    ax.set_aspect('equal')

    for triangulo in triangulos:
        x_values = [punto.x for punto in triangulo.puntos] + [triangulo.puntos[0].x]
        y_values = [punto.y for punto in triangulo.puntos] + [triangulo.puntos[0].y]
        ax.plot(x_values, y_values, 'k-')

    for punto in puntos:
        ax.plot(punto.x, punto.y, 'bo')

    ax.set_xlabel('Eje X')
    ax.set_ylabel('Eje Y')
    plt.show()

# Implementa la triangulación de Delaunay por fuerza bruta
def triangulacion_delaunay(puntos):
    triangulos = []  # Lista para almacenar los triángulos de Delaunay

    # Itera sobre todos los tripletes únicos de puntos
    for i in range(len(puntos)):
        for j in range(i + 1, len(puntos)):
            for k in range(j + 1, len(puntos)):
                p1, p2, p3 = puntos[i], puntos[j], puntos[k]
                centro, radio = calcular_circuncentro_y_radio(p1, p2, p3)

                if centro is None:
                    continue  # Omitir si los puntos son colineales

                # Verifica si algún otro punto está dentro de esta circunferencia
                es_delaunay = all(not punto_dentro_circunferencia(p, centro, radio)
                                  for p in puntos if p not in [p1, p2, p3])

                # Si es True, este triángulo cumple con la condición de Delaunay
                if es_delaunay:
                    triangulos.append(Triangulo([p1, p2, p3]))

    return triangulos

# Lectura de archivos
def cargar_puntos(archivo):
    puntos = []
    with open(archivo, 'r') as file:
        num_points = int(file.readline().strip())
        for line in file:
            x, y = map(float, line.strip().split())
            puntos.append(Punto(x, y))
    return puntos


### Codigo de la triangulacion

In [5]:

# ---- Inicio del código ---- #
prefix = "/Users/misa_v/Library/CloudStorage/OneDrive-InstitutoTecnologicoydeEstudiosSuperioresdeMonterrey/Sem 5/Algoritmos_/Scripts/Python/txt/"
archivos = ['puntos-n8.txt', 'puntos-n10.txt', 'puntos-n11.txt',
            'puntos-n15.txt', 'puntos-n16.txt', 'puntos-n20.txt',
            'puntos-n50.txt', 'puntos-n100.txt']

for archivo in archivos:
    puntos = cargar_puntos(prefix + archivo)
    triangulos = triangulacion_delaunay(puntos)
    ##graficar_triangulos_y_puntos(puntos, triangulos)


## Diagramas de Voronoy